In [1]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import nltk
import difflib
from sklearn.decomposition import LatentDirichletAllocation

pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [2]:
def show_card():
    while(1):
        card_name = input('Enter the name of a commander: ').strip().lower()
        if card_name=='exit':
            return
        try:
            card = commanders[commanders.index.str.lower()==card_name]
            response = requests.get(card.image_uris.tolist()[0]['small'])
            break
        except:
            print(f"Couldn't find {card_name}. Did you mean:")
            counter=1
            for name in difflib.get_close_matches(card_name.title(),commanders.index.tolist(),len(commanders.index.tolist()),0)[:5]:
                print(f"\t{counter}. {name}")
                counter+=1
    img = Image.open(BytesIO(response.content))
    return img

# Load cards and drop columns

In [3]:
commanders = pd.read_json('Data/scryfall-default-cards.json')
commanders.drop(columns=['all_parts','arena_id','artist_ids','booster','border_color','card_back_id','card_faces',
                       'collector_number','digital','flavor_text','foil','frame','frame_effects','full_art',
                       'hand_modifier','id','illustration_id','life_modifier','mtgo_foil_id','mtgo_id',
                       'multiverse_ids','nonfoil','object','oracle_id','printed_name','prints_search_uri',
                       'promo','promo_types','rarity','related_uris','rulings_uri','scryfall_set_uri','set_type',
                       'set_uri','story_spotlight','uri','variation','variation_of','watermark'],inplace=True)

In [4]:
commanders.columns

Index(['artist', 'cmc', 'color_identity', 'color_indicator', 'colors',
       'edhrec_rank', 'games', 'highres_image', 'image_uris', 'lang', 'layout',
       'legalities', 'loyalty', 'mana_cost', 'name', 'oracle_text',
       'oversized', 'power', 'preview', 'printed_text', 'printed_type_line',
       'released_at', 'reprint', 'reserved', 'scryfall_uri', 'set', 'set_name',
       'set_search_uri', 'tcgplayer_id', 'textless', 'toughness', 'type_line'],
      dtype='object')

**Only include legendary creatures**

In [5]:
commanders = commanders[commanders.type_line.str.contains('Legendary Creature')]

**Commander Legal only**

In [6]:
commanders.legalities = [card['commander'] for card in commanders.legalities]
commanders = commanders[commanders.legalities=='legal']

**Commander is meant to be a fun and social format, so drop digital-exclusive cards**

In [7]:
commanders.games = ['paper' in card for card in commanders.games]
commanders = commanders[commanders.games]

In [8]:
commanders = commanders[commanders.layout=='normal']

In [9]:
commanders = commanders[~commanders.oracle_text.str.contains('Partner')]

**Dataset includes reprints as a separate row, so drop those to have a unique list**

In [10]:
commanders.drop_duplicates('name','last',inplace=True)

**Set the index of my DataFrame to the card name**

In [11]:
commanders.set_index('name',inplace=True)

In [14]:
commanders.shape

(786, 31)

In [12]:
show_card()

Enter the name of a commander: exit


In [97]:
lda=LatentDirichletAllocation(n_components=10,random_state=0)

In [94]:
lda.fit(commanders.oracle_text.tolist())

ValueError: Expected 2D array, got 1D array instead:
array=['Vigilance, menace\nWhenever an equipped creature you control attacks, you draw a card and you lose 1 life.\nEquipment you control have equip Knight {0}.'
 'Flying\nWhenever Korvold, Fae-Cursed King enters the battlefield or attacks, sacrifice another permanent.\nWhenever you sacrifice a permanent, put a +1/+1 counter on Korvold and draw a card.'
 "Vigilance\nWhenever you cast a creature spell, draw a card, then you may put a land card from your hand onto the battlefield.\n{3}, {T}: Return target creature you control to its owner's hand."
 'Flying, deathtouch, lifelink\nOther creatures you control with flying get +1/+0.\nWhenever you cast an artifact or enchantment spell, create a 1/1 blue Faerie creature token with flying.'
 "{R}: All creatures gain trample and haste until end of turn.\n{1}{G}: Put a +1/+1 counter on target creature.\n{2}{W}: Target player gains 5 life.\n{3}{U}: Target player draws a card.\n{4}{B}: Put target